Moscow flats dataset, I will compare the accuracy of prediction of linear regression and of random forest

In [ ]:
# load necessary libraries
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error as mape

In [2]:
# download data
df = pd.read_csv('path_to_file')
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2040 entries, 0 to 2039
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2040 non-null   int64  
 1   price       2040 non-null   int64  
 2   totsp       2040 non-null   int64  
 3   livesp      2040 non-null   int64  
 4   kitsp       2040 non-null   float64
 5   dist        2040 non-null   float64
 6   metrdist    2040 non-null   int64  
 7   walk        2040 non-null   int64  
 8   brick       2040 non-null   int64  
 9   floor       2040 non-null   int64  
 10  code        2040 non-null   int64  
dtypes: float64(2), int64(9)
memory usage: 175.4 KB


,Unnamed: 0,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,code
count,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000
mean,1020.500000,127.496569,73.084314,46.337255,8.898529,11.015686,8.117157,0.685784,0.323039,0.790686,4.322059
std,589.041594,51.878220,15.123450,7.894348,2.787073,3.375539,3.815574,0.464317,0.467752,0.406918,2.183289
min,1.000000,50.000000,44.000000,28.000000,5.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,510.750000,95.000000,62.000000,42.000000,7.000000,9.000000,5.000000,0.000000,0.000000,1.000000,3.000000
50%,1020.500000,115.000000,73.500000,45.000000,9.000000,12.000000,7.000000,1.000000,0.000000,1.000000,4.000000
75%,1530.250000,142.000000,79.000000,50.000000,10.000000,13.500000,10.000000,1.000000,1.000000,1.000000,6.000000
max,2040.000000,730.000000,192.000000,102.000000,25.000000,17.000000,20.000000,1.000000,1.000000,1.000000,8.000000


There are no nan; 
Price - price of the flat in thousand US Dollars, totsp - square of flat in sq meters, livesp - living space in sq meters, kitsp - kitchen space in sq meters, dist - distance to city center in km, metrdist - distance to nearest metro station in min, walk - can we walk to metro? 1 or 0 , brick - 0 or 1, floor - 1 (if not the first of the last) or 0 - otherwise.

In [3]:
# drop unnecessary column
df = df.drop(columns='Unnamed: 0')

In [4]:
# divide data into test and train sample
train = df.sample(frac=0.8)
test = df.drop(train.index)

In [5]:
# divide data into dependent and independent variables
y = train.loc[:,'price']
x = train.loc[:,df.columns != 'price']
x_test = test.loc[:,df.columns != 'price']
y_test = test.loc[:,'price']

In [6]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [7]:
# Train the model on training data
rf.fit(x, y)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [8]:
# predict on test sample
predict_rf = rf.predict(x_test)

In [9]:
# regression model
x = sm.add_constant(x)
regr = OLS(y,x).fit()
print(regr.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.674
Model:                            OLS   Adj. R-squared:                  0.673
Method:                 Least Squares   F-statistic:                     373.3
Date:                Sat, 07 Jan 2023   Prob (F-statistic):               0.00
Time:                        19:14:44   Log-Likelihood:                -7830.1
No. Observations:                1632   AIC:                         1.568e+04
Df Residuals:                    1622   BIC:                         1.573e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -24.8852      6.271     -3.968      0.0

In [10]:
# regression prediction
x_test = sm.add_constant(x_test)
predict_regr = regr.predict(x_test)

In [11]:
# calculate errors
print('regression prediction error: ', mape(y_test, predict_regr))
print('random forest prediction error: ', mape(y_test, predict_rf))

regression prediction error:  0.13454103640542614
random forest prediction error:  0.10189937080808667


Random forest prediction shows more accuracy than that of regression, however both metrics are lower than 15%, thus both models can be applied, but random forest are generally better than regression in this particular case